# 1. Neural Network Basics

In [1]:
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

np.random.seed(10)

/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(ignore warning messages if you see any)

## 1.1 How a Neural Network Makes Predictions

In [2]:
# example input/response mapping 
def f(x):
    return np.where(np.dot(x, np.random.rand(3, 1)) > 0.8, 1., 0.)
    
X = np.random.random((1000, 3))
y_true = f(X)

# print(X[:3])
# print(y_true[:3])

Network topology:

<img src="images/Network.png"/>

In [3]:
# create the model
neural_network = Sequential()

In [4]:
hidden_layer1 = Dense(units=2, activation="relu", input_dim=3)
neural_network.add(hidden_layer1)

In [5]:
hidden_layer2 = Dense(units=3, activation='relu')
neural_network.add(hidden_layer2)

# output_layer = Dense(units=2, activation="sigmoid")
# neural_network.add(output_layer)

In [6]:
# summarize model architecture
neural_network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [7]:
# let's manually set the network's weights 
# (input neuron weights, bias weights)

## input --> hidden layer 1
random_weights1 = [np.random.random((3, 2)), np.ones(2)]
neural_network.layers[0].set_weights(random_weights1)

## hidden layer 1 --> hidden layer 2
random_weight2 = [np.random.random((2, 3)), np.ones(3)]
neural_network.layers[1].set_weights(random_weight2)

## hidden layer 2 --> output layer
# random_weight3 = [np.random.random((3, 2)), np.ones(2)]
# neural_network.layers[2].set_weights(random_weight3)

In [8]:
# predict y from a few X's
X_sample = X[:10,:]
y_sample_pred = neural_network.predict_classes(X_sample)
y_sample_true = y_true[:10]
print("Pred y:\t{}".format(y_sample_pred.reshape(-1)))
print("True y:\t{}".format(y_sample_true.reshape(-1)))

Pred y:	[0 0 0 0 0 0 0 0 0 0]
True y:	[1. 1. 0. 0. 0. 1. 1. 0. 1. 1.]


In [9]:
# evaluate model on predictive accuracy for all X
y_pred = neural_network.predict_classes(X)
acc = sum(y_true == y_pred.reshape(-1))[0]/float(y_true.shape[0])*100
print("Predictive accuracy: {}%".format(acc))

Predictive accuracy: 49.4%


## 1.2. How to Train a Neural Network

Network topology:

<img src="images/Network.png"/>

In [10]:
# create model
smart_neural_network = Sequential()

# smart_neural_network.add(Dense(2, activation='relu', input_dim=3))
# smart_neural_network.add(Dense(3, activation='relu'))
# smart_neural_network.add(Dense(2, activation='sigmoid'))


In [11]:
# define training parameters
smart_neural_network.compile(metrics=['accuracy'],
                             loss='binary_crossentropy',
                             optimizer='rmsprop')

In [12]:
# pre-processing: convert responses to 2d categorical vectors
from keras.utils import np_utils
y_categorical = np_utils.to_categorical(y_true)

<img src="images/GradientDescent.png" width=400/>

In [13]:
# train the model for a number of rounds
intial_weights = smart_neural_network.get_weights()
smart_neural_network.fit(X, y_categorical, 
                         epochs=100,
                         batch_size=32
)

Epoch 1/100
1000/1000 [==============================] - 1s 987us/step - loss: 0.6975 - acc: 0.4890
Epoch 2/100
1000/1000 [==============================] - 0s 158us/step - loss: 0.6936 - acc: 0.4815
Epoch 3/100
1000/1000 [==============================] - 0s 155us/step - loss: 0.6916 - acc: 0.5070
Epoch 4/100
1000/1000 [==============================] - 0s 149us/step - loss: 0.6903 - acc: 0.5265
Epoch 5/100
1000/1000 [==============================] - 0s 222us/step - loss: 0.6894 - acc: 0.5355
Epoch 6/100
1000/1000 [==============================] - 0s 142us/step - loss: 0.6886 - acc: 0.5430
Epoch 7/100
1000/1000 [==============================] - 0s 153us/step - loss: 0.6878 - acc: 0.5520
Epoch 8/100
1000/1000 [==============================] - 0s 174us/step - loss: 0.6870 - acc: 0.5555
Epoch 9/100
1000/1000 [==============================] - 0s 150us/step - loss: 0.6860 - acc: 0.5610
Epoch 10/100
1000/1000 [==============================] - 0s 217us/step - loss: 0.6849 - acc: 0.5705

In [14]:
print(smart_neural_network.metrics_names)

['loss', 'acc']


In [15]:
# evaluate the model accuracy on training data
score = smart_neural_network.evaluate(X, y_categorical)
print('Predictive accuracy on training data:', score[1])

1000/1000 [==============================] - 0s 243us/step
Predictive accuracy on training data: 0.97


### But how well can our neural network do on data it's never seen before?

In [16]:
# split data into training and validation sets
num_training_examples = 800

X_train = X[:num_training_examples]
y_train = y_categorical[:num_training_examples]
X_valid = X[num_training_examples:]
y_valid = y_categorical[num_training_examples:]

In [21]:
# re-train only on training examples
smart_neural_network.set_weights(intial_weights)

smart_neural_network.compile(metrics=['accuracy'], loss='binary_crossentropy', optimizer='rmsprop')
smart_neural_network.fit(X_train, y_train, batch_size=32, epochs=100)   

Epoch 1/100
800/800 [==============================] - 1s 940us/step - loss: 0.6982 - acc: 0.4713
Epoch 2/100
800/800 [==============================] - 0s 133us/step - loss: 0.6950 - acc: 0.4738
Epoch 3/100
800/800 [==============================] - 0s 115us/step - loss: 0.6929 - acc: 0.4756
Epoch 4/100
800/800 [==============================] - 0s 121us/step - loss: 0.6914 - acc: 0.4944
Epoch 5/100
800/800 [==============================] - 0s 120us/step - loss: 0.6904 - acc: 0.5156
Epoch 6/100
800/800 [==============================] - 0s 122us/step - loss: 0.6897 - acc: 0.5281
Epoch 7/100
800/800 [==============================] - ETA: 0s - loss: 0.6870 - acc: 0.560 - 0s 124us/step - loss: 0.6892 - acc: 0.5344
Epoch 8/100
800/800 [==============================] - 0s 131us/step - loss: 0.6887 - acc: 0.5406
Epoch 9/100
800/800 [==============================] - 0s 131us/step - loss: 0.6881 - acc: 0.5487
Epoch 10/100
800/800 [==============================] - 0s 144us/step - loss: 0.

800/800 [==============================] - 0s 123us/step - loss: 0.4067 - acc: 0.8588
Epoch 84/100
800/800 [==============================] - 0s 123us/step - loss: 0.4003 - acc: 0.8612
Epoch 85/100
800/800 [==============================] - 0s 128us/step - loss: 0.3944 - acc: 0.8650
Epoch 86/100
800/800 [==============================] - 0s 133us/step - loss: 0.3878 - acc: 0.8681
Epoch 87/100
800/800 [==============================] - 0s 120us/step - loss: 0.3808 - acc: 0.8694
Epoch 88/100
800/800 [==============================] - 0s 109us/step - loss: 0.3738 - acc: 0.8731
Epoch 89/100
800/800 [==============================] - 0s 105us/step - loss: 0.3670 - acc: 0.8744
Epoch 90/100
800/800 [==============================] - 0s 124us/step - loss: 0.3609 - acc: 0.8744
Epoch 91/100
800/800 [==============================] - 0s 118us/step - loss: 0.3543 - acc: 0.8769
Epoch 92/100
800/800 [==============================] - 0s 124us/step - loss: 0.3470 - acc: 0.8769
Epoch 93/100
800/800 [=

In [22]:
# evaluate on validation (never-seen-before) examples 
validation_scores = smart_neural_network.evaluate(X_valid, y_valid)
print('Predictive accuracy on validation data:', score[1])

200/200 [==============================] - 0s 820us/step
Predictive accuracy on validation data: 0.97


## Exercise: deep networks

Try creating a neural network model with a "deeper" architecture (e.g. more than just 1-2 hidden layers). Fit this on the training data from the input/response example given in the notebook and evaluate the predictive error. 

* Experiment with the number of neurons in each layer.
* When training, increase the number of epochs 

Do you experience any performance gains? Does training run slower than it did before? If so, by how much? Are the performance gains worth it? 

In [19]:
# TODO

<i>Sample solution:</i>

In [20]:
# deep_neural_network = Sequential()
# deep_neural_network.add(Dense(128, activation='relu', input_dim=3))
# deep_neural_network.add(Dense(64, activation='relu'))
# deep_neural_network.add(Dense(32, activation='relu'))
# deep_neural_network.add(Dense(2, activation='sigmoid'))
# deep_neural_network.compile(metrics=['accuracy'], loss='binary_crossentropy', optimizer='rmsprop')

# deep_neural_network.fit(X_train, y_train, epochs=100, batch_size=32)
# score = deep_neural_network.evaluate(X_valid, y_valid)

# print 'Predictive accuracy on validation data:', score[1]